In [117]:
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from collections import defaultdict 
from imdb import IMDb
import cPickle as pickle
import pandas as pd 
import io 
import time
import requests

### Scrape the movie gross Information from boxofficemojo.com 

In [99]:
years = xrange(2009,2015)
pages = xrange(1,9)
year_pagetxt = {}
for year in years: 
    pagestext = {}
    for page in pages: 
        r = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=%s&view=releasedate&view2=domestic&yr=%s&p=.htm"%(page, year))
        pagestext[page] = r.text
        time.sleep(1)
    year_pagetxt[year] = pagestext

In [100]:
movie_budget = defaultdict(list) 
for year in years: 
    for page in pages: 
        soup = BeautifulSoup(year_pagetxt[year][page], "html.parser")
        rows = soup.find_all("font", attrs={'size':'2'})
           
        start = 10 
        for i in range(start,len(rows)-2):
            t = rows[i].get_text()
            if unicode('Summary of') in t: 
                break
            elif (i-start) % 9 == 0: 
                movie_budget['rank'].append(t)
            elif (i-start) % 9 == 1: 
                movie_budget['year'].append(year)
                r = '('+str(year)
                if unicode(r) in t: 
                    j = t.index(unicode(r))
                    movie_budget['title'].append(t[:j])
                else: 
                    movie_budget['title'].append(t)
            elif (i-start) % 9 == 2: 
                movie_budget['studio'].append(t)
            elif (i-start) % 9 == 3: 
                movie_budget['gross'].append(t)
            elif (i-start) % 9 == 4: 
                movie_budget['gross theaters'].append(t)
            elif (i-start) % 9 == 5: 
                movie_budget['opening'].append(t)
            elif (i-start) % 9 == 6: 
                movie_budget['opening theaters'].append(t)
            elif (i-start) % 9 == 7: 
                movie_budget['open'].append(t)
            elif (i-start) % 9 == 8: 
                movie_budget['close'].append(t)

In [101]:
print len(movie_budget['title'])
print set(movie_budget['year'])

3715
set([2009, 2010, 2011, 2012, 2013, 2014])


###Scrape movie budget information from the-numbers.com 

In [102]:
r_numbers = requests.get("http://www.the-numbers.com/movie/budgets/all")

d_=pq(r_numbers.text)
d_tables=pq(d_('table'))
rows = pq(d_tables[0])('tr')

In [103]:
budget = defaultdict(list)
for j in range(1,len(rows)):
    dat = pq(rows[j])('td')
    for i in range(len(dat)):
        if i % 6 == 1:
            budget['year'].append(int(pq(dat[i])('a').text().split('/')[-1]))
        elif i % 6 == 2:
            t = pq(dat[i])('a').text()
            if 'Birdman' in t:
                budget['title'].append(t.split(' or ')[0])
            else:
                budget['title'].append(t)
        elif i % 6 == 3: 
            budget['budget'].append(pq(dat[i]).text())

### Search_movies might return many movies related to our search. This finds the appropriate movie.

In [104]:
def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            continue
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            #import ipdb;ipdb.set_trace()
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "").replace('(I)','')
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i].lower() == sorted_mtitle[i].lower():
                        counts[j] += 1
            else:
                continue
        
        if max(counts) <= len_sorted_title: 
            k = counts.index(max(counts))
        else: 
            k = counts.index(len_sorted_title)
        if len(year_list) >= 1:
            return year_list[k]
        else:
            return None

### Combine movie ids, budget info, and gross info into one dict

In [110]:
ia = IMDb(accessSystem='http')

#The next process actually attempts to match the movies in the Box Office dataframe to the 
BOmissingmovies = [] # To hold anything we don't find
BOdict = {} # This will actually contain the movies and information about them.

j = movie_budget['year'].index(2014)
print j
movienumber = len(movie_budget['year'])
print movienumber

3013
3715


In [115]:
for i in range(3013, 3715):
    movieobj = None
    # Need to process the row-level information out of BOdf in order to get the movie objects
    movieobj = ia.search_movie(movie_budget['title'][i])
    
    #Handling cases where we haven't found the movie or have multiples
    if movieobj is None or len(movieobj)>1:
        potential_movie_titles = ia.search_movie(movie_budget['title'][i])
        movieobj = find_movie(movie_budget['title'][i], movie_budget['year'][i], potential_movie_titles) # find the movie
        if type(movieobj) == list: 
            movieobj = movieobj[0]
            
    print (i, movie_budget['title'][i], movie_budget['year'][i], movieobj)
    
    if movieobj is not None and not (type(movieobj) == list and len(movieobj) == 0):
        ## Get movie id ##
        if type(movieobj) == list: 
            movieobj = movieobj[0]
        ia.update(movieobj)    
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        BOdict[movid] = {}
        # "title": title of movie
        BOdict[movid]['title'] = movie_budget['title'][i]
        BOdict[movid]['gross'] = movie_budget['gross'][i]
        BOdict[movid]['opening'] = movie_budget['opening'][i]
        BOdict[movid]['rank'] = movie_budget['rank'][i]
        BOdict[movid]['studio'] = movie_budget['studio'][i] 
        BOdict[movid]['open'] = movie_budget['open'][i]
        BOdict[movid]['close'] = movie_budget['close'][i]
        BOdict[movid]['opening theaters'] = movie_budget['opening theaters'][i]
        BOdict[movid]['year'] = movie_budget['year'][i]
        try:
            BOdict[movid]['rating'] = movieobj['rating']
        except: 
            BOdict[movid]['rating'] = []
        try:
            BOdict[movid]['mpaa'] = movieobj['mpaa']
        except: 
            BOdict[movid]['mpaa'] = []
        try:
            BOdict[movid]['director'] = movieobj['director']
        except: 
            BOdict[movid]['director'] = []
        try:
            BOdict[movid]['cast'] = movieobj['cast']
        except:
            BOdict[movid]['cast'] = []
        #BOdict[movid]['']
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
    else:
        BOmissingmovies.append((i, movie_budget['title'][i], movie_budget['year'][i]))
        

(3013, u'American Sniper', 2014, <Movie id:2179136[http] title:_American Sniper (2014)_>)
(3014, u'The Hunger Games: Mockingjay - Part 1', 2014, <Movie id:1951265[http] title:_The Hunger Games: Mockingjay - Part 1 (2014)_>)
(3015, u'Guardians of the Galaxy', 2014, <Movie id:2015381[http] title:_Guardians of the Galaxy (2014)_>)
(3016, u'Captain America: The Winter Soldier', 2014, <Movie id:1843866[http] title:_Captain America: The Winter Soldier (2014)_>)
(3017, u'The LEGO Movie', 2014, <Movie id:1490017[http] title:_The Lego Movie (2014)_>)
(3018, u'The Hobbit: The Battle of the Five Armies', 2014, <Movie id:2310332[http] title:_The Hobbit: The Battle of the Five Armies (2014)_>)
(3019, u'Transformers: Age of Extinction', 2014, <Movie id:2109248[http] title:_Transformers: Age of Extinction (2014)_>)
(3020, u'Maleficent', 2014, <Movie id:1587310[http] title:_Maleficent (2014)_>)
(3021, u'X-Men: Days of Future Past', 2014, <Movie id:1877832[http] title:_X-Men: Days of Future Past (2014)

In [123]:
df = pd.DataFrame.from_dict(BOdict.items())
df.set_index(0, inplace=True)
df.head()

KeyError: 0

KeyError: 0

In [124]:
pickle.dump(BOdict, io.open('BOdict_2014.p', 'wb'))
#BOdict = pickle.load(io.open('BOdict_2014.p', 'rb'))

In [125]:
# add seasons 
for k in BOdict.keys():
    open_date = BOdict[k]['open']
    if open_date:
        month = int(open_date.split('/')[0])
        if month <= 5 and month >= 1:
            BOdict[k]['season'] = 0
        elif month <= 9 and month >= 6:
            BOdict[k]['season'] = 1 
        elif month <= 11 and month >= 10:
            BOdict[k]['season'] = 2
        elif month == 12: 
            BOdict[k]['season'] = 3
        else:
            BOdict[k]['season'] = None 

In [63]:
# ids = [1418309, 3090538, 1477949, 1655246, 2173287, 1517098, 1534681, 5167030, 1427984, 1470490, 1434895, 1489965,1655236,1441109,1655223, 1410541, 1531545,1655251, 1617047, 1403193, 1079444, 3091388, 1444167,1527117, 1581548, 1626526,1032825,1398586,1523980,'1160368', '0840406','1466462', 1129441,1360866,1425244,1135503,'0892392','1924708',1773506,1344392,1172233,1826884,1492032,1623377,1485440,1342981,1575355,1549672,1523269,'0758751',1635532,1329410,1475953,1441246,1502518,3411658,1409712,1668035,1166113,2161644,1535923,1586084,2794428,1436614,2131963,2203271,1498858,2257807,1592097,3022534,1655245,3022266,1660057,1660837,1744171,5158556,1658403,3022286,2225433,2135110,1572311,1759480,'0473075',1434435,/
#        1690816,3990206,1843125,1757236,1589952,2327964,1767398,1883245,1817547,1599228,1914601,2160993,2784780,1759556,1684868,1687046,1664568,1732435,1785903,1610452,1795668,1367142,1817491,1356864,1853039,1629161,1747029,4316588,1715602,1758123,1676114,1758268,1891207,2355841,2329087,1695585,1769945,1594948,1702319,2135267,1861316,1609359,'0404034',2336484,1758472,1724147,1774553,1390533,1908154,2133584,2092975,1936888,3119228,1872255,2127928,3006306,2305455,2291369,2094238,1753798,3137876,1950928,1464540,2135158,2138451,1816730,1639078,1906398,2185451,2154723,2094761,2069441,3471070,1912979,2910956,1903228,2078565,1911420,2071063,2324817,1756427,2055721,1837654,2133191,1959186,1959186,2334109,1986096,/
#        1876446,3010880,2728428,2025163,1925084,1727491,2049991,'0458339',1555084,2329971,'0091830',2362103,1918665,1789308,1834948,2135288,1754919,1660379,1251743,1645774,2367954,1784600,2338120,2266200,2324805,1758606,1855801,2035569,2004372,1825963,2384455,2535508,2872906,2517380,2400910,2324845,3116178,3883116,2304986,3646580,2189383,3061264,3061310,3061318,2382632,2320509,1942316,3509488,2577770,2497934,2322137,2051747,2329851,3061230,2181589,2400004,2217904,2373627,3745846,2333408,2484076,2258241,1975249,1232200,1480656,2334562,2106476,3394488,1308729,2281527,2737494,2215215,2302519,2218564,2163336,3727004,2348492,2348492,2145992,2262456,2350940,2093853,2133239,2168650,2153342,2141717,2289242,2256265,/
       

{'gross': u'$515,005', 'rating': [], 'opening': u'$220,542', 'mpaa': [], 'opening theaters': u'102', 'title': u"A Hard Day's Night ", 'rank': u'252', 'director': [<Person id:6580067[http] name:_Sakellaropoulos, Andrea_>], 'cast': [<Person id:6580065[http] name:_AbbaKiss_>, <Person id:5265152[http] name:_Corten, Anthony_>, <Person id:4103436[http] name:_Dellamorto, Dixie_>, <Person id:1495195[http] name:_Lobo, Mister_>, <Person id:4394074[http] name:_Milliken, Jamie_>, <Person id:6580066[http] name:_Mittens, Miss_>, <Person id:6580063[http] name:_Sakura, George_>, <Person id:6580064[http] name:_Something, Richard_>], 'studio': u'Jan.', 'season': 1, 'close': u'7/20', 'open': u'7/4'}


In [138]:
print BOdict['2179136']

KeyError: 'budget'

In [136]:
for k in BOdict.keys()[:10]:
    for i in range(len(budget['title'])):
        budget_title = find_movie(budget['title'][i], BOdict[k]['year'], BOdict[k]['title'])
        if budget_title != None: 
            try:
                j  = budget['title'].index(budget_title)
                BOdict[k]['budget'] = budget['budget'][j]
                print BOdict[k]['budget']
            except:
                continue

In [137]:
# c=0 
# for k in BOdict.keys():
#     if BOdict[k]['budget']:
#         c+=1
# print c

KeyError: 'budget'